In [22]:
!pip install replicate


In [23]:
!pip install python-dotenv

In [80]:
!pip install openai --upgrade



  Using cached openai-1.33.0-py3-none-any.whl.metadata (21 kB)
Using cached openai-1.33.0-py3-none-any.whl (325 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [31]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 4.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install datasets


  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 856.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.3 MB/s eta 0:00:00 0:00:01
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5 kB 1.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 850.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [131]:
import os
from openai import OpenAI

#For testing

# Load environment variables
os.environ['OPENAI_API_KEY'] = 'add_key_here'

# Initialize OpenAI client
client = OpenAI()

# Create a chat completion
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": "Translate the following English text to French: 'Hello, how are you?'"
    }]
)

print(completion.choices[0].message.content)


Bonjour, comment vas-tu?


In [118]:
import openai
import replicate
import random
import pandas as pd
from datasets import load_dataset

# Load environment variables
openai_api_key = 'add_key_here'
replicate_api_key = 'add_key_here'

# Initialize OpenAI client
openai.api_key = openai_api_key


In [119]:
class LLM_Agent:
    def __init__(self, llm_model, model_type="openai", api_key=None):
        self.llm_model = llm_model
        self.model_type = model_type
        if self.model_type == "openai":
            if api_key:
                openai.api_key = api_key
            self.client = openai
        elif self.model_type == "replicate":
            if api_key:
                replicate_client = replicate.Client(api_token=api_key)
            self.client = replicate_client

    def ask_question(self, question):
        if self.model_type == "openai":
            completion = self.client.chat.completions.create(
                model=self.llm_model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": question}
                ],
                max_tokens=100
            )
            return completion.choices[0].message.content.strip()
            #did not have time to make the replicate work
        # elif self.model_type == "replicate":
        #     try:
        #         model = replicate.models.get(self.llm_model)
        #         version = model.versions.list()[0]
        #         inputs = {"prompt": question}
        #         output = version.predict(**inputs)
        #         return output['text'] if 'text' in output else output[0]
        #     except replicate.exceptions.ReplicateError as e:
        #         print(f"Replicate API error: {e}")
        #     except Exception as e:
        #         print(f"General error: {e}")

    def create_question_subset(self, dataset, num_questions=20):
        questions = dataset['validation']['question']
        if len(questions) < num_questions:
            print(f"Only {len(questions)} questions available. Using all available questions.")
            return list(questions)
        return random.sample(list(questions), num_questions)

    def run_experiment(self, questions):
        results = {}
        for question in questions:
            answer = self.ask_question(question)
            results[question] = answer
        return results

In [120]:
# Initialize the agents for different models
openai_agent = LLM_Agent(llm_model="gpt-3.5-turbo", model_type="openai")
# replicate_agent1 = LLM_Agent(llm_model="distilgpt-2", model_type="replicate")
# replicate_agent2 = LLM_Agent(llm_model="another-replicate-model", model_type="replicate")

# Load the MMLU dataset with a specific configuration
dataset = load_dataset('cais/mmlu', 'all')


In [127]:
# Instruct the agent to create a subset of 20 random questions
random_questions = openai_agent.create_question_subset(dataset, num_questions=20)

# Run the experiment for OpenAI agent
gpt_3_turbo__results = openai_agent.run_experiment(random_questions)

# Print the results for verification
print(gpt_3_turbo__results)


{'College students who live in dormitories near the college cafeteria are more likely to eat in the cafeteria. This is an example of': 'This is an example of the influence of **proximity** on behavior. In this case, the physical closeness of the dormitories to the college cafeteria increases the likelihood that students will choose to eat there. This concept highlights how environmental factors, such as location and accessibility, can significantly impact decision-making and habits.', "To celebrate its 100th anniversary, your company wants to publish a coffee-table book of photos depicting the company's history. You dig through the files and old annual reports and find many photos, some of which have never been published. Some of the unpublished photos are dated and stamped on the back with the name of a photography studio that has been out of business for 20 years. When you consult with your publisher about the most effective way to reproduce these particular prints, the publisher say

In [128]:
openai_agent = LLM_Agent(llm_model="gpt-4o", model_type="openai")
dataset = load_dataset('cais/mmlu', 'all')
gpt_4o_results = openai_agent.run_experiment(random_questions)
print(gpt_4o_results)

{'College students who live in dormitories near the college cafeteria are more likely to eat in the cafeteria. This is an example of': 'This is an example of the influence of **proximity** on behavior. Proximity refers to the physical closeness or distance between places or people. In this case, students who live in dormitories near the college cafeteria are more likely to eat there because it is conveniently located and requires less effort to access. This concept is often studied in psychology and sociology to understand how physical environments can impact human behavior and decision-making.', "To celebrate its 100th anniversary, your company wants to publish a coffee-table book of photos depicting the company's history. You dig through the files and old annual reports and find many photos, some of which have never been published. Some of the unpublished photos are dated and stamped on the back with the name of a photography studio that has been out of business for 20 years. When yo

In [116]:
# This cell uses another model to assess the output of the previous model

# Define a function to assess the answers
def assess_answers_with_another_model(answers, assessment_model="gpt-4", api_key=None):
    if api_key:
        openai.api_key = api_key
    client = openai.OpenAI()

    assessment_results = {}
    for question, answer in answers.items():
        assessment_prompt = f"Assess the following answer for correctness:\n\nQuestion: {question}\nAnswer: {answer}\n\nIs this answer correct? Explain why or why not."
        assessment = client.chat.completions.create(
            model=assessment_model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": assessment_prompt}
            ],
            max_tokens=150
        )
        assessment_results[question] = assessment.choices[0].message.content.strip()
    return assessment_results

# Example usage: Assess the answers provided by openai_agent using another model
assessment_results = assess_answers_with_another_model(gpt_3_turbo__results, assessment_model="gpt-4", api_key='add_key_here')

# Print the assessment results
print("Assessment of the answers:")
for question, assessment in assessment_results.items():
    print(f"Q: {question}\nAssessment: {assessment}\n")


Assessment of the answers:
Q: Which of the following statements that relate to capital budgeting is true?
Assessment: Yes, the answer is correct. The statement accurately describes one aspect of capital budgeting, which is the process of evaluating and selecting long-term investment opportunities to generate returns over an extended period of time. Capital budgeting is a critical financial management tool used by companies to make informed decisions about where to allocate their financial resources for long-term growth and profitability. So, the answer is accurate in explaining the essence of capital budgeting.

Q: If the point $(3,6)$ is on the graph of $y=g(x)$, and $h(x)=(g(x))^2$ for all $x$, then there is one point that must be on the graph of $y=h(x)$. What is the sum of the coordinates of that point?
Assessment: The answer provided is not entirely correct. While the process of calculating $h(3)$ as $(g(3))^2$ and obtaining $h(3) = 6^2 = 36$ is correct, the answer does not provid

In [132]:
# Example usage: Assess the answers provided by openai_agent using another model
assessment_results = assess_answers_with_another_model(gpt_4o_results, assessment_model="gpt-4", api_key='add_key_here')

# Print the assessment results
print("Assessment of the answers:")
for question, assessment in assessment_results.items():
    print(f"Q: {question}\nAssessment: {assessment}\n")


Assessment of the answers:
Q: College students who live in dormitories near the college cafeteria are more likely to eat in the cafeteria. This is an example of
Assessment: Yes, the answer is correct. The response accurately defines "proximity" and demonstrates how it is applied in the given scenario. It outlines how the convenience and ease of access resulting from the physical closeness of the dormitories to the cafeteria influences students' decisions to eat there, accurately relating it to how the physical environment can impact human behavior. This concept is indeed widely studied in fields like psychology and sociology.

Q: To celebrate its 100th anniversary, your company wants to publish a coffee-table book of photos depicting the company's history. You dig through the files and old annual reports and find many photos, some of which have never been published. Some of the unpublished photos are dated and stamped on the back with the name of a photography studio that has been out 